In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint
from segmentation_models import get_preprocessing
import segmentation_models as sm
import os

gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

Segmentation Models: using `keras` framework.


In [2]:
import os
import cv2
import numpy as np
import tensorflow as tf

# Define paths to images and masks
IMAGE_PATH = r'Dataset\images'
MASK_PATH = r'Dataset\masks'
CLASSES = ['Astrocytoma', 'Glioblastoma', 'Gliomas', 'Meningioma', 'Neurocytoma']
NUM_CLASSES = len(CLASSES)

# Create a mapping from class names to integer labels
class_mapping = {class_name: index for index, class_name in enumerate(CLASSES)}

def load_image_mask(image_path, mask_path, class_name):
    # Load image
    image = cv2.imread(image_path.decode('utf-8'), cv2.IMREAD_COLOR)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # Load mask
    mask = cv2.imread(mask_path.decode('utf-8'), cv2.IMREAD_GRAYSCALE)
    
    # Normalize image to [0, 1]
    image = image / 255.0
    
    # Normalize mask to [0, 1]
    mask = mask / 255.0
    
    # One-hot encode the mask
    mask = (mask > 0).astype(np.uint8)  # Convert to binary mask (0 or 1)
    mask_one_hot = np.zeros((mask.shape[0], mask.shape[1], NUM_CLASSES), dtype=np.uint8)
    mask_one_hot[..., class_mapping[class_name.decode('utf-8')]] = mask
    
    return image, mask_one_hot

def load_dataset(image_dir, mask_dir):
    image_paths = []
    mask_paths = []
    labels = []
    
    for class_name in CLASSES:
        image_class_dir = os.path.join(image_dir, class_name)
        mask_class_dir = os.path.join(mask_dir, class_name)
        
        for image_filename in os.listdir(image_class_dir):
            image_path = os.path.join(image_class_dir, image_filename)
            mask_path = os.path.join(mask_class_dir, image_filename)
            
            image_paths.append(image_path)
            mask_paths.append(mask_path)
            labels.append(class_name)
    
    return image_paths, mask_paths, labels

def process_data(image_path, mask_path, label):
    image, mask = load_image_mask(image_path.numpy(), mask_path.numpy(), label.numpy())
    return image, mask

def tf_process_data(image_path, mask_path, label):
    image, mask = tf.py_function(process_data, [image_path, mask_path, label], [tf.float32, tf.uint8])
    image.set_shape([None, None, 3])
    mask.set_shape([None, None, NUM_CLASSES])
    return image, mask

image_paths, mask_paths, labels = load_dataset(IMAGE_PATH, MASK_PATH)

# Create a TensorFlow dataset
dataset = tf.data.Dataset.from_tensor_slices((image_paths, mask_paths, labels))
dataset = dataset.map(lambda img_path, msk_path, lbl: tf_process_data(img_path, msk_path, lbl))

# Batch and prefetch the dataset for training
batch_size = 8
dataset = dataset.batch(batch_size)
dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)

# Verify the dataset by iterating and printing shapes



In [3]:
filepath = "checkpoints/improvement - {val_accuracy:.2f}.keras"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', save_best_only= True, mode = 'max')
callbacks_list = [checkpoint]

In [4]:
BACKBONE = "inceptionresnetv2"

model = sm.Unet(BACKBONE, classes=5, activation='softmax')
model.compile(
    'Adam',
    loss=sm.losses.bce_jaccard_loss,
    metrics=[sm.metrics.iou_score],
)

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None, None,  0           []                               
                                 3)]                                                              
                                                                                                  
 conv2d (Conv2D)                (None, None, None,   864         ['input_1[0][0]']                
                                32)                                                               
                                                                                                  
 batch_normalization (BatchNorm  (None, None, None,   96         ['conv2d[0][0]']                 
 alization)                     32)                                                           

In [5]:
training = model.fit_generator(dataset, epochs=10, callbacks=callbacks_list)

Epoch 1/10


C:\Users\Lenevo\AppData\Local\Temp\ipykernel_30632\3565153568.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  training = model.fit_generator(dataset, epochs=10, callbacks=callbacks_list)


TypeError: in user code:

    File "c:\Users\Lenevo\anaconda3\envs\tensorflowgpu\lib\site-packages\keras\engine\training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "c:\Users\Lenevo\anaconda3\envs\tensorflowgpu\lib\site-packages\segmentation_models\base\objects.py", line 114, in __call__  *
        return self.l1(gt, pr) + self.l2(gt, pr)
    File "c:\Users\Lenevo\anaconda3\envs\tensorflowgpu\lib\site-packages\segmentation_models\losses.py", line 130, in __call__  *
        return F.binary_crossentropy(gt, pr, **self.submodules)
    File "c:\Users\Lenevo\anaconda3\envs\tensorflowgpu\lib\site-packages\segmentation_models\base\functional.py", line 256, in binary_crossentropy  *
        return backend.mean(backend.binary_crossentropy(gt, pr))
    File "c:\Users\Lenevo\anaconda3\envs\tensorflowgpu\lib\site-packages\keras\backend.py", line 5677, in binary_crossentropy
        return tf.nn.sigmoid_cross_entropy_with_logits(

    TypeError: Input